## Projet connaissances et raisonnement

## Modélisation du problème

Nous voulons trouver **un créneau optimal** pour une réunion en fonction des disponibilités des participants.


In [ ]:
import os
import csv
from itertools import combinations

# Configuration
SCHEDULES_DIR = r"schedules"      # Dossier des emplois du temps
DURATION = 60                  # Durée en minutes (15, 30, 45, 60)
GOPHERSAT_PATH = r"gophersat.exe"  # chemin vers le solveur

## 1. Lecture des emplois du temps

In [ ]:
def lire_emplois_du_temps(dossier):
    """
    Lit tous les fichiers CSV dans le dossier et retourne :
      - emplois : un dictionnaire où chaque clé correspond à un participant et
                  la valeur est un dictionnaire {jour: {horaire: statut}}
                  Le statut vaut 0 si le participant est disponible, 1 sinon.
      - jours : la liste des jours considérés
      - horaires : la liste des horaires (extrait du header des CSV)
    """
    fichiers = [f for f in os.listdir(dossier) if f.endswith(".csv")]
    if not fichiers:
        raise FileNotFoundError("Aucun fichier CSV trouvé dans le dossier")
    jours = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
    emplois = {}  # clé : identifiant du participant, valeur : emploi du temps
    horaires = None
    for fichier in fichiers:
        with open(os.path.join(dossier, fichier), 'r', encoding='utf-8') as f:
            lecteur = csv.reader(f)
            header = next(lecteur)
            # La première colonne est "Jour", le reste sont les horaires
            if horaires is None:
                horaires = header[1:]
            emploi = {}  # emploi du temps pour ce participant
            for ligne in lecteur:
                jour = ligne[0]
                if jour not in jours:
                    continue
                # Pour chaque jour, créer un dictionnaire {horaire: statut}
                emploi[jour] = {}
                for i, statut in enumerate(ligne[1:]):
                    # Convertir le statut en entier (0 pour libre, 1 pour occupé)
                    emploi[jour][horaires[i]] = int(statut)
            # On peut utiliser le nom du fichier comme identifiant du participant
            emplois[fichier] = emploi
            
            # S'assurer que tous les jours et horaires sont présents
            for jour in jours:
                if jour not in emploi:
                    emploi[jour] = {}
                for h in horaires:
                    if h not in emploi[jour]:
                        # Par défaut, on considère que le participant est disponible
                        emploi[jour][h] = 0
                        
    return emplois, jours, horaires

## 2. Génération du fichier CNF

In [ ]:
def generer_fichier_cnf_complet(emplois, jours, horaires, duree):
    """
    Génère un fichier CNF (au format DIMACS) pour la planification d'une réunion.
    
    Modélisation :
      - Pour chaque participant p, jour d et horaire s, on définit une variable X_{p,d,s}.
      - Pour chaque candidat de réunion (début de bloc) Y_{d,s}, pour s tel que le bloc de longueur m (duree//15)
        tient dans la liste des horaires, on définit une variable Y_{d,s}.
    
    Contraintes :
      1. Exactement un bloc de réunion est choisi (variable Y) :
           - Au moins un bloc choisi.
           - Pas deux blocs simultanément (exclusion mutuelle).
      2. Lien entre Y et X :
           Pour chaque candidat Y_{d,s} et pour chaque participant p,
           et pour chaque créneau du bloc (s, s+1, ..., s+m-1), on impose :
             Y_{d,s} -> X_{p,d,t}   (soit, en CNF : -Y_{d,s} v X_{p,d,t})
      3. Disponibilité des participants :
           Pour chaque participant p, jour d, horaire s, si le participant est occupé
           (selon l'emploi du temps), alors X_{p,d,s} est forcé à faux.
      4. Interdiction des blocs traversant la pause de midi.
      5. Limitation aux horaires de travail (8h-18h).
    """
    m = duree // 15  # Nombre de créneaux consécutifs requis

    clauses = []
    next_var = 1

    # 1. Création des variables X : X_vars[(participant, jour, horaire)]
    X_vars = {}
    for participant in emplois:
        for d in jours:
            for t in horaires:
                X_vars[(participant, d, t)] = next_var
                next_var += 1

    # 2. Création des variables Y : Y_vars[(jour, horaire_debut)]
    Y_vars = {}
    for d in jours:
        # Seul les candidats qui permettent d'avoir m créneaux consécutifs
        for i in range(len(horaires) - m + 1):
            t_debut = horaires[i]
            # Vérifier que le créneau est dans les horaires de travail (8h-18h)
            if "08:00" <= t_debut <= "17:45":  # Pour une réunion se terminant à 18h au plus tard
                Y_vars[(d, t_debut)] = next_var
                next_var += 1
    

    # ======================
    # Contrainte (1) : Exactement un bloc réunion est choisi (variables Y)
    # a) Au moins un bloc choisi :
    clause = " ".join(str(Y_vars[(d, t)]) for (d, t) in Y_vars) + " 0"
    clauses.append(clause)
    # b) Au plus un bloc choisi : pour chaque paire distincte, pas de double sélection
    for (d1, t1), (d2, t2) in combinations(Y_vars.keys(), 2):
        clauses.append(f"-{Y_vars[(d1, t1)]} -{Y_vars[(d2, t2)]} 0")

    # ======================
    # Contrainte (2) : Lien entre Y et X
    # Si le bloc Y_{d,t_debut} est choisi, alors pour chaque participant et pour chaque créneau
    # du bloc (de t_debut à t_debut+m-1), le participant doit être disponible (X_{p,d,t} doit être vrai)
    for (d, t_debut) in Y_vars:
        start_index = horaires.index(t_debut)
        # On s'assure que le bloc s'étend sur m créneaux
        for participant in emplois:
            for j in range(m):
                t = horaires[start_index + j]
                # Clause: -Y_{d,t_debut} v X_{p,d,t}
                clauses.append(f"-{Y_vars[(d, t_debut)]} {X_vars[(participant, d, t)]} 0")

    # ======================
    # Contrainte (3) : Disponibilité des participants (selon leur emploi du temps)
    # Pour chaque participant, jour, horaire, si le participant est occupé (statut == 1)
    # alors la variable X_{p,d,t} est forcée à faux
    for participant, emploi in emplois.items():
        for d in jours:
            for t in horaires:
                if emploi[d].get(t, 0) == 1:  # Utilisation de get() pour gérer les valeurs manquantes
                    # Le participant n'est pas disponible à ce créneau
                    clauses.append(f"-{X_vars[(participant, d, t)]} 0")
                    
    # ======================
    # Contrainte (4) : Interdiction des blocs traversant la pause de midi
    for (d, t_debut) in Y_vars:
        start_index = horaires.index(t_debut)
        # Vérifier si le bloc dépasse la pause de midi
        for j in range(m):
            t = horaires[start_index + j]
            if t_debut < "12:00" and t >= "14:00":
                clauses.append(f"-{Y_vars[(d, t_debut)]} 0")
                break  # Pas besoin de vérifier les autres créneaux, le bloc est interdit
            
        # Vérifier aussi si le bloc commence avant la pause et se termine après
        if start_index + m - 1 < len(horaires):
            t_fin = horaires[start_index + m - 1]
            if t_debut < "12:00" and t_fin > "12:00":
                clauses.append(f"-{Y_vars[(d, t_debut)]} 0")

    # ======================
    # Écriture du fichier CNF
    total_vars = next_var - 1
    total_clauses = len(clauses)
    with open("reunion.cnf", "w") as f:
        f.write(f"p cnf {total_vars} {total_clauses}\n")
        for clause in clauses:
            f.write(clause + "\n")

    print(f"Fichier CNF généré avec {total_vars} variables et {total_clauses} clauses.")
    return True, Y_vars, total_vars, total_clauses



## 3. Résolution et interprétation

In [ ]:
def resoudre_toutes_les_solutions(Y_vars, total_vars, total_clauses):
    solutions = []
    exclusion_clauses = []
    
    while True:
        # Update CNF file header
        with open("reunion.cnf", "r") as f:
            lines = f.readlines()
        
        updated_header = f"p cnf {total_vars} {total_clauses + len(exclusion_clauses)}\n"
        lines[0] = updated_header
        
        with open("reunion.cnf", "w") as f:
            f.writelines(lines)
            for clause in exclusion_clauses:
                f.write(clause + "\n")
        
        os.system(f"{GOPHERSAT_PATH} reunion.cnf > resultat.txt")

        solution = []
        with open("resultat.txt", "r") as f:
            for ligne in f:
                if ligne.startswith("v "):
                    solution = list(map(int, ligne.strip().split()[1:-1]))
                elif "UNSATISFIABLE" in ligne:
                    return solutions

        if not solution:
            break

        reunion_cree = [var for var in solution if var > 0 and var in Y_vars.values()]
        if not reunion_cree:
            break
            
        solutions.append(solution)
        exclusion_clause = " ".join(f"-{var}" for var in reunion_cree) + " 0"
        exclusion_clauses.append(exclusion_clause)

    return solutions

In [ ]:
def afficher_resultats_detailles(solutions, Y_vars, emplois, jours, horaires, duree):
    if not solutions:
        print("Aucune solution trouvée.")
        return

    print(f"\n=== Créneaux possibles pour une réunion de {duree} minutes ===\n")
    
    for i, solution in enumerate(solutions, 1):
        reunion_var = [var for var in solution if var > 0 and var in Y_vars.values()]
        if not reunion_var:
            continue
            
        for (jour, horaire), var in Y_vars.items():
            if var in reunion_var:
                m = duree // 15
                start_index = horaires.index(horaire)
                horaire_fin = horaires[start_index + m - 1]
                h, min = map(int, horaire_fin.split(':'))
                min += 15
                if min >= 60:
                    h += 1
                    min -= 60
                horaire_fin_reel = f"{h:02d}:{min:02d}"

                print(f"Option {i}: {jour} {horaire}-{horaire_fin_reel}")
                
                tous_disponibles = True
                indisponibles = []
                for participant, emploi in emplois.items():
                    if not all(emploi[jour].get(horaires[start_index + j], 0) == 0 for j in range(m)):
                        tous_disponibles = False
                        indisponibles.append(participant)
                
                if tous_disponibles:
                    print("✅ Tous les participants sont disponibles")
                else:
                    print("❌ Participants non disponibles:", ", ".join(indisponibles))
                print()
                break

## 4. Exécution principale

In [ ]:
emplois, jours, horaires = lire_emplois_du_temps(SCHEDULES_DIR)
print(f"Créneaux occupés chargés pour {len(emplois)} participants")
success, Y_vars, total_vars, total_clauses = generer_fichier_cnf_complet(emplois, jours, horaires, DURATION)


In [ ]:
if success:
    solutions = resoudre_toutes_les_solutions(Y_vars, total_vars, total_clauses)
    afficher_resultats_detailles(solutions, Y_vars, emplois, jours, horaires, DURATION)
else:
    print("Erreur lors de la génération du fichier CNF.")